## Without TOP-K sampling

In [1]:
import torch
def generatenextword(model,idx,max_token_size,context_Size):

    for _ in range(max_token_size):
        idx_split=idx[:,-context_Size:]

        with torch.no_grad():
            logit=model(idx_split)

        ## logit [ batch tokens vocabsize]
        logitslast=logit[:,-1,:]

        prob=torch.softmax(logitslast,dim=-1)
        maxi=torch.argmax(prob,dim=-1,keepdim=True)
        idx=torch.cat((idx,maxi),dim=1)
    return idx  



## With TOP-K sampling

In [3]:
next_token_logits = torch.tensor(
[4.51, 0.89, -1.90, 6.75, 1.63, -1.62, -1.89, 6.28, 1.79]
)



top_k = 3
top_logits, top_pos = torch.topk(next_token_logits, top_k)
print("Top logits:", top_logits)
print("Top positions:", top_pos)
new_logits = torch.where(
    condition=next_token_logits < top_logits[-1],
    input=torch.tensor(float("-inf")), 
    other=next_token_logits
)

print(new_logits)


Top logits: tensor([6.7500, 6.2800, 4.5100])
Top positions: tensor([3, 7, 0])
tensor([4.5100,   -inf,   -inf, 6.7500,   -inf,   -inf,   -inf, 6.2800,   -inf])
tensor([7])


## merging TOP-K with TEMP SCALING

In [ ]:
import torch

def generatenextword(model, idx, max_token_size, context_Size, top_k=5, temperature=1.0):
    for _ in range(max_token_size):
        idx_split = idx[:, -context_Size:]

        with torch.no_grad():
            logits = model(idx_split)

        logits_last = logits[:, -1, :]
        
        logits_last = logits_last / temperature
        
       
        values, indices = torch.topk(logits_last, k=top_k, dim=-1)
        probs = torch.softmax(values, dim=-1)
        sampled_idx = torch.multinomial(probs, num_samples=1)
        maxi = torch.gather(indices, -1, sampled_idx)
       
    
        idx = torch.cat((idx, maxi), dim=1)
    
    return idx
